In [1]:
##pip install pandas pyarrow psycopg2-binary

In [38]:
import pandas as pd

df = pd.read_parquet('data/organized/2024-02/yellow_tripdata_2024-02.parquet')

import numpy as np
df.replace({np.nan: None}, inplace=True)

print(df.head(10))


   VendorID tpep_pickup_datetime tpep_dropoff_datetime passenger_count  \
0         2  2024-02-01 00:04:45   2024-02-01 00:19:58             1.0   
1         2  2024-02-01 00:56:31   2024-02-01 01:10:53             1.0   
2         2  2024-02-01 00:07:50   2024-02-01 00:43:12             2.0   
3         1  2024-02-01 00:01:49   2024-02-01 00:10:47             1.0   
4         1  2024-02-01 00:37:35   2024-02-01 00:51:15             1.0   
5         1  2024-02-01 00:55:17   2024-02-01 01:01:53             1.0   
6         2  2024-02-01 00:04:53   2024-02-01 00:17:07             1.0   
7         2  2024-02-01 00:35:00   2024-02-01 00:43:37             1.0   
8         2  2024-02-01 00:00:15   2024-02-01 00:05:07             1.0   
9         2  2024-02-01 00:25:55   2024-02-01 00:31:52             1.0   

  trip_distance RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0          4.39        1.0                  N            68           236   
1          7.71        1.0     

In [ ]:
import psycopg2
import urllib.parse

dbname = 'nyc'
user = 'postgres'
password = 'password'
host = 'localhost'  # or your database server IP
port=5433

conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

In [40]:
cursor = conn.cursor()

# 3. Define the INSERT statement matching the EXACT order of your DataFrame columns
insert_statement = """
INSERT INTO green_tripdata_2024_01 (
    VendorID,
    lpep_pickup_datetime,
    lpep_dropoff_datetime,
    store_and_fwd_flag,
    RatecodeID,
    PULocationID,
    DOLocationID,
    passenger_count,
    trip_distance,
    fare_amount,
    extra,
    mta_tax,
    tip_amount,
    tolls_amount,
    ehail_fee,
    improvement_surcharge,
    total_amount,
    payment_type,
    trip_type,
    congestion_surcharge
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# 4. Insert each row
#    Here we assume df columns are in EXACT order:
#    ['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'store_and_fwd_flag', 
#     'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 
#     'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 
#     'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge']
for row in df.itertuples(index=False, name=None):
    try:
        cursor.execute(insert_statement, row)
        print
    except Exception as e:
        print(f"Row failed: {row} - {e}")
        conn.rollback()  # Clears the aborted state
        # optionally skip or transform the bad row
        # continue  # then move on

# 5. Commit and close
conn.commit()
cursor.close()
conn.close()
print("Data inserted successfully!")

Data inserted successfully!
